<a href="https://colab.research.google.com/github/Arunsh-Data/ArunshAI.github.io/blob/main/Project_Sales_Dashboard_EDA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd


In [2]:
url = '/content/sample_data/Sales Dashboard Dataset_01.xlsx'
df = pd.read_excel(url)

print(df.head())

FileNotFoundError: [Errno 2] No such file or directory: '/content/sample_data/Sales Dashboard Dataset_01.xlsx'

In [ ]:
xls = pd.ExcelFile(url)
xls.sheet_names

In [ ]:
# Select the Sales sheet
sales_df = xls.parse('Sales')

# Perform EDA on the sales sheet
print("\n--- EDA for Sales Sheet ---")
print("\nShape:")
print(sales_df.shape)
print("\nInfo:")
print(sales_df.info())
print("\nDescribe:")
print(sales_df.describe())
print("\nMissing values:")
print(sales_df.isnull().sum())

# Select the Customer sheet
customer_df = xls.parse('Customers')

# Perform EDA on the customer sheet
print("\n--- EDA for Customers Sheet ---")
print("\nShape:")
print(customer_df.shape)
print("\nInfo:")
print(customer_df.info())
print("\nDescribe:")
print(customer_df.describe())
print("\nMissing values:")
print(customer_df.isnull().sum())

In [ ]:
# Check for duplicate rows in the Sales sheet
print("\nDuplicate rows in Sales sheet:")
print(sales_df.duplicated().sum())

# Check for duplicate rows in the Customers sheet
print("\nDuplicate rows in Customers sheet:")
print(customer_df.duplicated().sum())

In [ ]:
# Merge the Sales and Customers sheets on the 'CustomerID' column
merged_df = pd.merge(sales_df, customer_df, on='Customer ID')

# Display the first few rows of the merged DataFrame to verify
print("\n--- Merged DataFrame ---")
print(merged_df.head())

# Display information about the merged DataFrame
print("\nMerged DataFrame Info:")
print(merged_df.info())

In [ ]:
merged_df['Full Name'] = merged_df['Customer First Name_x'] + ' ' + merged_df['Customer Last Name_x']
merged_df = merged_df.drop(['Customer First Name_x', 'Customer Last Name_x', 'Customer First Name_y', 'Customer Last Name_y'], axis=1)

print("\n--- DataFrame with Full Name and dropped First/Last Name ---")
print(merged_df.head())
print("\nInfo after adding Full Name and dropping First/Last Name:")
print(merged_df.info())

In [ ]:
# Identify date columns - Assuming columns with 'Date' in their name are date columns
date_columns = [col for col in merged_df.columns if 'Date' in col]

# Iterate through identified date columns and format them
for col in date_columns:
    # Ensure the column is in datetime format first (important if not already)
    merged_df[col] = pd.to_datetime(merged_df[col], errors='coerce')
    # Format the date column to 'mm-year'
    merged_df[col] = merged_df[col].dt.strftime('%m-%Y')

# Display the head of the DataFrame with formatted date columns
print("\n--- Merged DataFrame with formatted date columns ---")
print(merged_df.head())

# Display info to confirm the data type (will likely be 'object' after formatting)
print("\nInfo after formatting date columns:")
print(merged_df.info())

# Calculate Key Performance Indicators (KPIs)

In [ ]:
# Calculate Total Revenue
total_revenue = merged_df['Sale Price _x'].sum()
print(f"Total Revenue: ${total_revenue:,.2f}")

# Calculate Total Profit
total_profit = merged_df['Total Profit (GMROI)'].sum()
print(f"Total Profit: ${total_profit:,.2f}")

# Calculate Number of Orders
number_of_orders = merged_df['Order ID'].nunique()
print(f"Number of Orders: {number_of_orders}")

# Calculate Average Order Value
average_order_value = total_revenue / number_of_orders
print(f"Average Order Value: ${average_order_value:,.2f}")

In [ ]:
writer = pd.ExcelWriter('kpi_dashboard.xlsx', engine='xlsxwriter')

# Create a DataFrame for KPIs
kpi_data = {
    'KPI': ['Total Revenue', 'Total Profit', 'Number of Orders', 'Average Order Value'],
    'Value': [total_revenue, total_profit, number_of_orders, average_order_value]
}
kpi_df = pd.DataFrame(kpi_data)

# Write the KPIs DataFrame to a sheet named 'KPIs'
kpi_df.to_excel(writer, sheet_name='KPIs', index=False)

# You might want to write other dataframes (like the original sheets or the merged_df) to other sheets
# Example:
sales_df.to_excel(writer, sheet_name='Original_Sales', index=False)
merged_df.to_excel(writer, sheet_name='Merged_Data', index=False)


# Save the Excel file
writer.close()

print("\nKPIs and other data written to kpi_dashboard.xlsx")

In [ ]:
xls_kpi = pd.ExcelFile('kpi_dashboard.xlsx')
print("Sheets in kpi_dashboard.xlsx:")
print(xls_kpi.sheet_names)